Copyright 2023 Province of British Columbia

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at 

   http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under he License.
the License.

## Imports and Parameters

In [1]:
# add our stuff to the path
import sys
import os
sys.path.insert(1, os.path.join(sys.path[0], '..'))

# other stuff
import time
import pandas as pd
import numpy as np

pd.set_option('display.max_colwidth', None)

# import our stuff
from importlib import reload
from src import connect, preprocess, synthetic, model

In [2]:
# PARAMETERS

# info to gain access to database, IDIR restricted 
CRED_PATH = '../credentials.txt'

# where model is stored. requires credentials.txt for full path 
MODEL_BASE_PATH = 'Model/Q22'

# which tables to access
RESPONSE_TABLE = 'dbo.AQ22ANCES'
CODE_TABLE = 'dbo.AQ22ANCES_Codes'
CLOSED_TABLE = 'dbo.Q22ANCESMultiResponse'
RESULTS_TABLE = 'dbo.AQ22ANCES_RESULTS'

# which column to use/create 
RESPONSE_COLUMN = 'aq22_cleaned'
OUTPUT_COLUMNS = 'q22ances_c'
N_COLUMNS = 5

# delimiter to send back with concatenated results
DELIMITER = 'μ' 

# amount of synthetic data to produce
N_PER_CATEGORY = 10_000
N_SYNTHETIC_MIXED = 100_000

# threshold for accepting as a flagged category
THRESHOLD = 0.5

# upper and lower thresholds for flagging as a possible category
TENTATIVE_UPPER = 0.75
TENTATIVE_LOWER = 0.25

## Read from Database

In [3]:
# Read in all data required to build model
connection = connect.create_connection(CRED_PATH)

# actual responses
df_open = connect.fetch_table(RESPONSE_TABLE, connection)

# codes to match
code_df = connect.fetch_table(CODE_TABLE, connection)

# closed respones (for multi response frequencies)
df_closed = connect.fetch_table(CLOSED_TABLE, connection)

C:\Users\lforeste\Work\dev\github_repos\bcgov\demo-nlp\notebooks\..\src\connect.py:70: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f'SELECT * FROM {table_name}', connection)


In [4]:
# Reshaping the dataframe

# do this piece wise to deal with weird shape 
df_reshaped = pd.DataFrame(
    columns = ['id', 'cycle', 'q22ances', 'aq22ances'] + 
                [f'q22ances_c0{ii}' for ii in range(1, 6)] +
                ['origin']
)

for jj in range(1, 5+1):
    cols = ['id', 'cycle', f'q22ances{jj}', f'aq22ances{jj}'] + [f'q22ances_c{(jj-1)*5 + ii:02}' for ii in range(1 ,6)]
    df_tmp = df_open.loc[:, cols]
    df_tmp['origin'] = jj
    df_tmp.columns = df_reshaped.columns
    df_reshaped = pd.concat([df_reshaped, df_tmp])

# Filtering rows where q22ances and aq22ances are not None
df_reshaped = df_reshaped[df_reshaped['q22ances'].notna() & df_reshaped['aq22ances'].notna()]
df_reshaped = df_reshaped.sort_values(by='id')

In [5]:
df_reshaped[df_reshaped.cycle == 1].head()

,id,cycle,q22ances,aq22ances,q22ances_c01,q22ances_c02,q22ances_c03,q22ances_c04,q22ances_c05,origin
1127,0000004,1,97,74% North Western European,42,None,None,None,None,4
7177,0000007,1,97,Germany,44,None,None,None,None,4
7238,0000011,1,24µ97,brazil,24,None,None,None,None,2
1445,0000013,1,97,"Scottland, England & Germany",42,44,None,None,None,4
7425,0000015,1,97,Kenya,14,None,None,None,None,1


## Preprocess Data

* Use only cycle 1 for training.
* Use the cleaned column.
* Correct spelling where possible (takes longer time to run).
* Create input and output tables for model (input table takes a while to produce)

In [6]:
# for training purposes, open responses should only be those from cycle 1
df = df_reshaped[df_reshaped.cycle == 1].reset_index(drop=True)
df.head()

,id,cycle,q22ances,aq22ances,q22ances_c01,q22ances_c02,q22ances_c03,q22ances_c04,q22ances_c05,origin
0,0000004,1,97,74% North Western European,42,None,None,None,None,4
1,0000007,1,97,Germany,44,None,None,None,None,4
2,0000011,1,24µ97,brazil,24,None,None,None,None,2
3,0000013,1,97,"Scottland, England & Germany",42,44,None,None,None,4
4,0000015,1,97,Kenya,14,None,None,None,None,1


In [7]:
# Use the cleaned column if available; otherwise, use the original column
df.loc[:, RESPONSE_COLUMN] = df.apply(
    lambda x: str(x.aq22ances).lower() if x.aq22ances is not None else np.nan, 
    axis=1
)

# Now only apply correct_spelling on non-null values
mask = df[RESPONSE_COLUMN].notnull()
df.loc[mask, RESPONSE_COLUMN] = df.loc[mask, RESPONSE_COLUMN].apply(preprocess.correct_spelling)

df.head()

,id,cycle,q22ances,aq22ances,q22ances_c01,q22ances_c02,q22ances_c03,q22ances_c04,q22ances_c05,origin,aq22_cleaned
0,0000004,1,97,74% North Western European,42,None,None,None,None,4,74% north western european
1,0000007,1,97,Germany,44,None,None,None,None,4,germany
2,0000011,1,24µ97,brazil,24,None,None,None,None,2,brazil
3,0000013,1,97,"Scottland, England & Germany",42,44,None,None,None,4,"scotland, england & germany"
4,0000015,1,97,Kenya,14,None,None,None,None,1,kenya


In [8]:
reload(preprocess)

<module 'src.preprocess' from 'C:\\Users\\lforeste\\Work\\dev\\github_repos\\bcgov\\demo-nlp\\notebooks\\..\\src\\preprocess.py'>

In [9]:
# get long form table of codes: step 1: turn countries into rows
code_df_long_tmp = preprocess.get_long_form_codes(code_df)
code_df_long_tmp.head(20)

,code,code_desc,description
0,88,I'm not able to be more specific,i'm not able to be more specific
1,99,Prefer not to answer,prefer not to answer
2,80000,Comment,comment
3,11,North Africa,tunisia
4,11,North Africa,western sahara
5,11,North Africa,libyan arab jamahiriya
6,11,North Africa,morocco
7,11,North Africa,north africa
8,11,North Africa,egypt
9,11,North Africa,algeria


In [10]:
# try it without step 2?
code_df_long = code_df_long_tmp.copy()

In [11]:
# step 2: add additional rows for each country (nationalities, other spellings, etc.)
code_df_long = preprocess.get_long_form_codes_q22(code_df_long_tmp)

In [12]:
# build a training dataset for the model 
# first part of dataset: actual data
code_df_long['code'] = code_df_long['code'].astype(str)
df[RESPONSE_COLUMN] = df[RESPONSE_COLUMN].astype(str)

# INPUTS TO MODEL
headers = list(preprocess.get_scores('test', code_df_long, as_df = True).col_id.values)

input_df = preprocess.get_scores_from_df(df, RESPONSE_COLUMN, code_df_long, headers=headers)
display(input_df.head())

,response,80000_comment_ratio,11_tunisia_ratio,11_tunisian_ratio,11_western sahara_ratio,11_sahrawi_ratio,11_moroccan_ratio,11_morocco_ratio,11_north africa_ratio,11_egypt_ratio,...,54_cook islands_set,54_tonga_set,54_tongan_set,54_futuna islands_set,54_polynesia_set,54_french polynesian_set,54_french polynesia_set,201_middle east_set,202_hawaii_set,100_indeterminate_set
0,74% north western european,18,18,24,50,18,29,18,42,13,...,27,13,19,26,24,38,34,28,13,32
1,germany,29,14,27,29,29,40,14,21,33,...,21,33,46,19,25,25,26,22,15,40
2,brazil,0,15,14,20,46,29,15,33,0,...,22,18,17,30,13,17,18,35,33,21
3,"scotland, england & germany",24,18,23,34,24,23,12,31,19,...,33,21,27,42,30,39,35,29,13,38
4,kenya,33,33,31,32,17,15,0,24,40,...,24,40,36,21,29,36,38,25,18,33


In [13]:
# OUTPUTS OF MODEL
# converts the coded columns into wide form 1/0 binary responses for every option 
output_df = preprocess.get_outputs_wide(df, RESPONSE_COLUMN, code_df_long, OUTPUT_COLUMNS, N_COLUMNS)
output_df.head()

,response,80000,11,12,13,14,15,21,22,23,...,42,43,44,51,52,53,54,201,202,100
0,74% north western european,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,germany,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,brazil,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"scotland, england & germany",0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
4,kenya,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Create Synthetic Data

* To augment our training data, produce synthetic data from the available phrases in the code list. 
* Both singular phrases and multi-response phrases are produced.
* The multi-response phrases are randomly generated according to weights associated with the non-written responses. 

In [14]:
reload(synthetic)

<module 'src.synthetic' from 'C:\\Users\\lforeste\\Work\\dev\\github_repos\\bcgov\\demo-nlp\\notebooks\\..\\src\\synthetic.py'>

In [16]:
# create synthetic data
# this section will create synthetic data that matches a single category based on available phrases 
input_columns = list(input_df.columns)
output_columns = list(output_df.columns)

extra_input_df, extra_output_df = synthetic.create_single_phrase_synthetic(
    output_df, 
    input_columns,
    output_columns,
    code_df_long,
    n_per_category = N_PER_CATEGORY,
    use_given = False
)


Code: 42 -- Observations: 0 + 10000. Done.
Code: 44 -- Observations: 0 + 10000. Done.
Code: 41 -- Observations: 0 + 10000. Done.
Code: 43 -- Observations: 0 + 10000. Done.
Code: 21 -- Observations: 0 + 10000. Done.
Code: 32 -- Observations: 0 + 10000. Done.
Code: 33 -- Observations: 0 + 10000. Done.
Code: 34 -- Observations: 0 + 10000. Done.
Code: 24 -- Observations: 0 + 10000. Done.
Code: 23 -- Observations: 0 + 10000. Done.
Code: 35 -- Observations: 0 + 10000. Done.
Code: 12 -- Observations: 0 + 10000. Done.
Code: 15 -- Observations: 0 + 10000. Done.
Code: 14 -- Observations: 0 + 10000. Done.
Code: 11 -- Observations: 0 + 10000. Done.
Code: 100 -- Observations: 0 + 10000. Done.
Code: 201 -- Observations: 0 + 10000. Done.
Code: 31 -- Observations: 0 + 10000. Done.
Code: 22 -- Observations: 0 + 10000. Done.
Code: 80000 -- Observations: 0 + 10000. Done.
Code: 13 -- Observations: 0 + 10000. Done.
Code: 52 -- Observations: 0 + 10000. Done.
Code: 51 -- Observations: 0 + 10000. Done.
Code:

In [17]:
# create synthetic data
# this section will create synthetic data that matches multiple categories
mixed_input_df, mixed_output_df = synthetic.create_multi_phrase_synthetic(
    output_df,
    df_closed,
    input_columns,
    output_columns,
    code_df_long,
    N_SYNTHETIC_MIXED
)

100000/100000  |---------------------------------------------------------------------------------------------------->|
Creating synthetic data outputs... Done.
Creating synthetic data inputs... Done.


In [18]:
# Concatenate with existing data
final_input_df = pd.concat(
    [
        #input_df, 
        extra_input_df, 
        mixed_input_df
    ], 
    ignore_index=True).drop('response', axis=1).astype(int)
final_output_df = pd.concat(
    [
        #output_df, 
        extra_output_df, 
        mixed_output_df
    ]
    , 
    ignore_index=True).drop('response', axis=1).astype(int)

In [ ]:
final_input_df.shape

In [ ]:
final_output_df.shape

## Train Model

To deal with the fact we have many possible categorical outputs, use a simple random forest model that will handle multiple outputs better than other models.

* After training, save model. 

In [ ]:
reload(model)

In [19]:
n_estimators = 100
clf = model.create_model(final_input_df, final_output_df, n_estimators = n_estimators)

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed: 14.3min


In [33]:
model.save_model(CRED_PATH, MODEL_BASE_PATH, clf, code_df_long)

In [ ]:
clf, code_df_long = model.load_model(CRED_PATH, MODEL_BASE_PATH)

## Produce Model Results

* Look at some results by hand to see if it makes sense
* Create results for entire hand-coded dataset
* Send results back to databse

In [42]:
preprocess.get_scores(sentence, code_df_long, as_df=True).sort_values(by='value', ascending=False).head(20)

,col_id,value
740,32_china_partial,100
1720,32_china_set,100
750,33_singaporean_partial,67
751,33_singapore_partial,67
730,32_republic of korea_partial,64
1708,32_south korea_set,64
736,32_north korean_partial,64
735,32_north korea_partial,64
728,32_south korea_partial,64
727,32_south korean_partial,64


In [37]:
sentence = 'china korea'
model.list_classes(sentence, code_df_long, clf, spellcheck=True)


TOP MATCHES FOR: china korea
CORRECTED TO:    china korea

35.00% (42)
Northern Europe

29.00% (32)
East Asia

24.00% (43)
Southern Europe

23.00% (33)
Southeast Asia

22.00% (44)
Western Europe

15.00% (41)
Eastern Europe

10.00% (34)
South Asia

8.00% (35)
West Asia

5.00% (21)
North America



[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s


In [ ]:
results_df = model.produce_results(
    df, input_df, output_df, 
    clf,
    threshold=THRESHOLD,
    tentative_lower = TENTATIVE_LOWER,
    tentative_upper = TENTATIVE_UPPER,
    delimiter = DELIMITER
)

In [ ]:
# save back to database
engine = connect.create_connection(CRED_PATH, sqlalchemy=True)

# for initial save of cycle 1, always replace. any subsequent inputs should be appended
connect.save_table(results_df, RESULTS_TABLE, engine, how='replace')